In [1]:
import numpy as np
from numpy import array as arr

In [2]:
def hardy_cross_M(L, D): 
    lamda = 0.021 / pow(D, 0.3)
    return 0.0826 * lamda * pow(L, 2) / pow(D, 5)

def hardy_cross_numerator(M, Q, n = 2):
    return M*Q*pow(abs(Q), n-1)

def hardy_cross_denominator(M, Q, n = 2):
    return n*M*pow(abs(Q), n-1)

def hardy_cross_flow_correction(numerator, denominator):
    return -1* numerator/denominator


def hardy_cross_loop_cor(pipes_M, pipes_Q, pipes_name = None, loop_name = None ):
    num_sum = 0
    den_sum = 0
    for i in range(len(pipes_Q)):
        num_sum += hardy_cross_numerator(M = pipes_M[i], Q = pipes_Q[i])
        den_sum += hardy_cross_denominator(M = pipes_M[i], Q = pipes_Q[i])
        
    delta_Q = hardy_cross_flow_correction(num_sum, den_sum)
    
    print(f'delta Q for {loop_name} = {delta_Q * pow(10, 3)}')
    return delta_Q

def v(Q, D):
    return 4*Q/np.pi*pow(D,2)

In [3]:
Qin = 58

Q = {'a': 10 + 1, #litru/secunda
     'b': 12 + 1,
     'c': 8.5 + 1,
     'd': 11 + 1,
     'e': 7.5 + 1,
     'q_dist': 0.01, #litru/ (secunda x metru)
     }
diametre_vane = arr([200, 200, 125, 125, 100, 125]) *pow(10, -3)

loop_names = ['loop 1', 'loop 2', 'loop 3']
loop_0_flow = arr([28, 1, -4, -28]) * pow(10, -3)
loop_1_flow = arr([15, 2, 3.5 , -1])* pow(10, -3)
loop_2_flow = arr([-3.5, -11, 4])   * pow(10, -3)

D_0 = arr([150 ,50 ,75 ,150])* pow(10, -3)
D_1 = arr([125, 75, 50, 50]) * pow(10, -3)
D_2 = arr([75, 50, 125])     * pow(10, -3)

L_0 = arr([200, 150, 350, 200])
L_1 = arr([400, 200, 250, 150])
L_2 = arr([350, 250, 100])

M_0 = hardy_cross_M(L = L_0 , D = D_0) 
M_1 = hardy_cross_M(L = L_1 , D = D_1)
M_2 = hardy_cross_M(L = L_2 , D = D_2)

names_0 = arr(['1-2', '2-6', '6-5', '5-1'])
names_1 = arr(['2-3', '3-4', '4-6', '6-2'])    
names_2 = arr(['6-4', '4-5', '5-6',])    

In [4]:
delta_Q_med = 1
k = 0
while (Qin)/1000  < delta_Q_med:
    delta_Q_med = 0
    print (f'ITERATION {k+1}')
    k += 1
    
    #delta_Qs
    delta_Q0 = hardy_cross_loop_cor(M_0, loop_0_flow, names_0, loop_name = loop_names[0] )
    delta_Q1 = hardy_cross_loop_cor(M_1, loop_1_flow, names_1, loop_name = loop_names[1])   
    delta_Q2 = hardy_cross_loop_cor(M_2, loop_2_flow, names_2, loop_name = loop_names[2])     

    #loop corections
    loop_0_flow += delta_Q0
    loop_1_flow += delta_Q1  
    loop_2_flow += delta_Q2  
    
    loop_0_flow[1] -= delta_Q1
    loop_0_flow[2] -= delta_Q2
    
    loop_1_flow[2] -= delta_Q2
    loop_1_flow[3] -= delta_Q0
    
    loop_2_flow[0] -= delta_Q1
    loop_2_flow[2] -= delta_Q0 
    print('\n')
    
    loops = [loop_0_flow, loop_1_flow, loop_2_flow ]
    names = [names_0, names_1, names_2]
    
    #afisaje
    for i, loop in enumerate(loops):
        for j, flow in enumerate(loop):
            print(f'{loop_names[i]}, {names[i][j]}, {flow  * pow(10, 3)}')
        print('\n')
    
    delta_Q_med = abs(delta_Q0) + abs(delta_Q1) + abs(delta_Q2)
    delta_Q_med = delta_Q_med* pow(10, 3) /3
    print(delta_Q_med)
    print(Qin/1000,'\n')

ITERATION 1
delta Q for loop 1 = 1.1942413192762946
delta Q for loop 2 = -1.9346904163182563
delta Q for loop 3 = 5.242733969478447


loop 1, 1-2, 29.194241319276294
loop 1, 2-6, 4.128931735594551
loop 1, 6-5, -8.048492650202153
loop 1, 5-1, -26.805758680723706


loop 2, 2-3, 13.065309583681742
loop 2, 3-4, 0.06530958368174372
loop 2, 4-6, -3.6774243857967037
loop 2, 6-2, -4.128931735594551


loop 3, 6-4, 3.6774243857967037
loop 3, 4-5, -5.757266030521552
loop 3, 5-6, 8.048492650202153


2.7905552350243332
0.058 

ITERATION 2
delta Q for loop 1 = 1.2258509048083723
delta Q for loop 2 = 1.4976413808804558
delta Q for loop 3 = 2.2681551132902067


loop 1, 1-2, 30.420092224084666
loop 1, 2-6, 3.8571412595224683
loop 1, 6-5, -9.090796858683987
loop 1, 5-1, -25.579907775915334


loop 2, 2-3, 14.5629509645622
loop 2, 3-4, 1.5629509645621997
loop 2, 4-6, -4.447938118206455
loop 2, 6-2, -3.857141259522468


loop 3, 6-4, 4.447938118206454
loop 3, 4-5, -3.4891109172313453
loop 3, 5-6, 9.09079685

In [5]:
def zeta(P, D, Q, ro = 997, g = 9.81):
    return P*pow(D,4)/(0.0826 * ro * g *pow(Q,2))

In [6]:
P6 = 11 * 9806.65 #mH2O --> PA
ro = 997
g = 9.81

P2 = P6 + M_0[1]*pow(loop_0_flow[1],2)/ro*g
P4 = P6 + M_1[2]*pow(loop_1_flow[2],2)/ro*g
P5 = P6 + M_2[0]*pow(loop_2_flow[0],2)/ro*g
P3 = P4 + M_1[1]*pow(loop_1_flow[1],2)/ro*g
P1 = P2 + M_0[0]*pow(loop_0_flow[0],2)/ro*g


presiuni_noduri = arr([P1, P2, P3, P4, P5, P6])
delta_p = presiuni_noduri - P6
H = delta_p/(ro*g)

debite_iesiri = [Qin]
debite_iesiri += [value for value in Q.values() if value > 1] #adauga debitul intrare in 1
debite_iesiri = np.array(debite_iesiri)

zetas = zeta(P = delta_p, D = diametre_vane, Q = debite_iesiri)

print(presiuni_noduri, '\n')
print(delta_p, '\n')
print(H, '\n')
print(zetas, '\n')

[107950.67907657 107930.32595671 107933.78251987 107907.79440872
 107881.06781868 107873.15      ] 

[77.52907657 57.17595671 60.63251987 34.64440872  7.91781868  0.        ] 

[0.00792685 0.00584587 0.00619928 0.00354217 0.00080955 0.        ] 

[4.56440733e-08 9.35844817e-07 1.08421335e-07 1.16006355e-07
 6.80610817e-09 0.00000000e+00] 

